# 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用PyTorch提供的接口更方便地实现线性回归的训练。

## 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [1]:
import torch

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.normal(mean=torch.zeros(labels.shape), std=0.01)

## 读取数据

Pytorch的`utils`中提供了`data`包来读取数据。由于`data`常用作变量名，我们将导入的`data`模块用添加了torch首字母的假名`tdata`代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [2]:
from torch.utils import data as tdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = tdata.TensorDataset(features, labels)
# 随机读取小批量
data_iter = tdata.DataLoader(dataset, batch_size, shuffle=True)

这里`data_iter`的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本。

In [3]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.0255, -1.3971],
        [ 2.1460, -1.1916],
        [ 0.6476, -0.1836],
        [-1.2027, -0.7906],
        [-1.4211, -0.9525],
        [ 0.2168, -1.3676],
        [ 0.0313,  1.6980],
        [-0.9016,  0.3065],
        [ 0.4038,  1.4425],
        [ 0.8925,  0.9993]]) tensor([ 9.0113, 12.5377,  6.1233,  4.4782,  4.5965,  9.2797, -1.5120,  1.3541,
         0.0972,  2.5914])


## 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，nn提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用nn更简洁地定义线性回归。

首先，导入`nn`模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。我们先定义一个模型变量`net`，它是一个`Sequential`实例。在nn中，`Sequential`实例可以看作是一个串联各个层的容器。在构造模型时，我们在该容器中依次添加层。当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。

In [4]:
from torch import nn

net = nn.Sequential()

回顾图3.1中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。在nn中，全连接层是一个`Linear`实例。该层输入特征数为2，输出个数为1。

In [5]:
net.add_module('linear', nn.Linear(2, 1))

## 初始化模型参数

在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。我们从nn导入`init`模块。该模块提供了模型参数初始化的各种方法。这里的`init`是`initializer`的缩写形式。我们通过`init.normal(tensor, std=0.01)`指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。

**因为PyTorch没有直接提供对整个模型参数进行初始化的接口，这里自定义函数初始化整个模型的参数**

In [6]:
from torch.nn import init

def params_init(model):
    if isinstance(model, nn.Linear):
        init.normal_(tensor=model.weight.data, std=0.01)
        init.constant_(tensor=model.bias.data, val=0)

net.apply(params_init)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

## 定义损失函数

在PyTorch中，`nn`模块定义了各种损失函数。我们直接使用它提供的均方损失作为模型的损失函数。

In [7]:
loss = nn.MSELoss() # 均方损失等于平方损失除以样本数

## 定义优化算法

同样，我们也无须实现小批量随机梯度下降。在导入optim后，我们创建一个`SGD`实例，并指定学习率为0.03的小批量随机梯度下降（`sgd`）为优化算法。该优化算法将用来迭代`net`实例所有通过`add_module`函数嵌套的层所包含的全部参数。这些参数可以通过`parameters`函数获取。

In [8]:
from torch import optim

optimizer = optim.SGD(net.parameters(), lr=0.03)

## 训练模型

在使用PyTorch训练模型时，我们通过调用`optimizer`实例的`step`函数来迭代模型参数。

In [9]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        net.zero_grad()
        l = loss(net(X), y.reshape(batch_size, -1))
        l.backward()
        optimizer.step()
    with torch.no_grad():
        l = loss(net(features), labels.reshape(num_examples, -1))
        print('epoch %d, loss: %f' % (epoch, l.data.numpy()))

epoch 1, loss: 0.000241
epoch 2, loss: 0.000099
epoch 3, loss: 0.000099


下面我们分别比较学到的模型参数和真实的模型参数。我们从`net`获得需要的层，并访问其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [10]:
linear = net[0]
true_w, linear.weight.data

([2, -3.4], tensor([[ 1.9997, -3.3999]]))

In [11]:
true_b, linear.bias.data

(4.2, tensor([4.2007]))

## 小结

* 使用PyTorch接口可以更简洁地实现模型。
* 在PyTorch中，`data`模块提供了有关数据处理的工具，`nn`模块定义了大量神经网络的层以及各种损失函数。
* PyTorch的`init`模块提供了模型参数初始化的各种方法。


## 练习

* 查阅PyTorch文档，看看`nn`和`init`模块里提供了哪些损失函数和初始化方法。
* 如何访问`linear.weight`的梯度？




## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/742)

![](../img/qr_linear-regression-gluon.svg)